# enums (enumerations)

`enums` give you a way of saying a value is one of a possible set of values. For example, we may want to say that Rectangle is one of a set of possible shapes that also includes Circle and Triangle. Or, a Coin is one of: Penny, Nickel, Dime or Quarter

# match

`match` is a control flow construct that allows you to compare a value against a series of patterns and then execute code based on which pattern matches the value. Patterns can be made up of literal values, variable names, etc.

### enum and match example

Below is an example of a `enum` on a set of types of coins. 

This enum is used by a function `value_in_cents` that takes a coin and uses `match` to determine which coin type it is and returns that type's value in cents.

A match has arms, the first arm is `Coin::Penny => 1,`. An arm has two parts: a pattern and some code. The first arm has a pattern that is the value `Coin::Penny` and then the `=>` operator that separates the pattern and the code to run. The code in this case is just the value `1`. Each arm is separated from the next with a comma `,`

to run multiple lines of code in a match arm, you must use curly brackets, and the comma following the arm is then optional.

enum variants can hold data inside them and match arms can bind to that data. see the old vs new quarters below

In [30]:
#[derive(Debug)] // so we can inspect the state in a minute

enum NewOld {
    New,
    Old
}

enum Coin {
    Penny,
    Nickel,
    Dime,
    Quarter(NewOld),
}

fn value_in_cents(coin: Coin) -> u8 {
    match coin {
        Coin::Penny => {
            println!("Lucky penny!");
            1
        }
        Coin::Nickel => 5,
        Coin::Dime => 10,
        Coin::Quarter(state) => {
            println!("quarter of type {:?}", state);
            25
        }
    }
}

fn main() {
    let nickel = Coin::Nickel;
    println!("{}", value_in_cents(nickel));

    let p = Coin::Penny;
    println!("{}", value_in_cents(p));

    let Q = Coin::Quarter(NewOld::Old);
    println!("{}", value_in_cents(Q));
}

main();

5
Lucky penny!
1
quarter of type Old
25


### Enum variant types

You can put any kind of data inside an enum variant: strings, numeric types, or structs, for example. You can even include another enum! Also, standard library types are often not much more complicated than what you might come up with.

Wanting to store IP addresses and encode which kind they are is so common that the standard library has a `IpAddr` definition.

Note that even though the standard library contains a definition for IpAddr, we can still create and use our own definition without conflict because we haven’t brought the standard library’s definition into our scope, which is made of structs for each IP address type.

In [2]:
enum IpAddr {
    V4(u8, u8, u8, u8),
    V6(String),
}

let home = IpAddr::V4(127, 0, 0, 1);

let loopback = IpAddr::V6(String::from("::1"));

In this example, the `Message` enum has four variants with different types:

1. Quit has no data associated with it at all.
2. Move has named fields, like a struct does.
3. Write includes a single String.
4. ChangeColor includes three i32 values.

We define the `call` method on the `Message` enum. The body of the `call` method uses `&self` to get the value that we called the method on. For example, `m1` has the value `Message::Write(String::from("hello"))`, and that is what `&self` will be in the body of the call method `m.call()`.  

In [6]:
enum Message {
    Quit,
    Move { x: i32, y: i32 },
    Write(String),
    ChangeColor(i32, i32, i32),
}

impl Message {
    fn call(&self) {
        match self {
            Message::Quit => println!("Quit"),
            Message::Move { x, y } => println!("Move to x: {}, y: {}", x, y),
            Message::Write(s) => println!("Write: {}", s),
            Message::ChangeColor(r, g, b) => println!("Change color to RGB({}, {}, {})", r, g, b),
        }
    }
}

fn main() {
    let m1 = Message::Write(String::from("hello"));
    m1.call();
    
    let m2 = Message::Move { x: 10, y: 20 };
    m2.call();
    
    let m3 = Message::ChangeColor(255, 0, 0);
    m3.call();
}

main();

Write: hello
Move to x: 10, y: 20
Change color to RGB(255, 0, 0)


# The `Option` `Enum`

The problem with null values is that if you try to use a null value as a not-null value, you’ll get an error of some kind. Because this null or not-null property is pervasive, it’s extremely easy to make this kind of error.

However, the concept that null is trying to express is still a useful one: a null is a value that is currently invalid or absent for some reason.

The problem isn’t really with the concept but with the particular implementation. As such, Rust does not have nulls, but it does have an enum that can encode the concept of a value being present or absent. This enum is Option<T>, and it is defined by the standard library as follows:

```rust
enum Option<T> {
    None,
    Some(T), // the Some variant
}
```

The `<T>` syntax is a generic type parameter, `<T>` means that the `Some` variant of the Option enum can hold one piece of data of any type, and that each concrete type that gets used in place of T makes the overall Option<T> type a different type. 

You dont ever have to actually write that out `Option`, but it is always there for you to use like this:

In [9]:
/*
For absent_number, Rust requires us to annotate the overall Option type: 
the compiler can’t infer the type that the corresponding Some variant will hold by looking only at a None value. 
Here, we tell Rust that we mean for absent_number to be of type Option<i32>.
*/

let absent_number: Option<i32> = None; 

let some_number = Some(5); // The type of some_number is Option<i32> 
let some_char = Some('e'); // The type of some_char is Option<char>

`Option<T>` is better than having null because `Option<T>` and `T` (where T can be any type) are different types. The compiler won’t let us use an `Option<T>` value as if it were definitely a valid value. For example, this code won’t compile, because it’s trying to add an `i8` to an `Option<i8>`:

In [10]:
let x: i8 = 5;
let y: Option<i8> = Some(5);

let sum = x + y;

Error: consider importing one of these items

Error: cannot add `Option<i8>` to `i8`

Only when we have an Option<i8> (or whatever type of value we’re working with) do we have to worry about possibly not having a value, and the compiler will make sure we handle that case before using the value.

In other words, you have to convert an Option<T> to a T before you can perform T operations with it. Generally, this helps catch one of the most common issues with null: assuming that something isn’t null when it actually is.

```
$ cargo run
   Compiling enums v0.1.0 (file:///projects/enums)
error[E0277]: cannot add `Option<i8>` to `i8`
 --> src/main.rs:5:17
  |
5 |     let sum = x + y;
  |                 ^ no implementation for `i8 + Option<i8>`
  |
  = help: the trait `Add<Option<i8>>` is not implemented for `i8`
  = help: the following other types implement trait `Add<Rhs>`:
            <&'a i8 as Add<i8>>
            <&i8 as Add<&i8>>
            <i8 as Add<&i8>>
            <i8 as Add>

For more information about this error, try `rustc --explain E0277`.
error: could not compile `enums` due to previous error
```

Below:

`let result = value.map(|x| x + 1).unwrap_or(0);` means

`value.map(|x| x + 1)` The map method is used on the Option type. If the Option contains a value (Some), the closure (|x| x + 1) is applied to the inner value (x), and a new Option with the result is returned. If the Option is None, the map operation is skipped, and None is returned. In this case, the closure adds 1 to the inner integer if it exists.

`.unwrap_or(0)` The unwrap_or method is then called on the Option. If the Option is Some, it returns the inner value. If the Option is None, it returns the provided default value, which is 0 in this case. This ensures that even if the original Option is None, we get a default value of 0.

So, the entire line is essentially saying: "If value is Some, add 1 to the inner integer; otherwise, if value is None, use 0 as the result." The final result is stored in the variable result.

In [22]:
:dep rand
use rand::Rng;

fn main() {
    // Import the necessary trait for generating random numbers
    let mut rng = rand::thread_rng();

    // Generate a random boolean to determine if the value should be an integer or None
    let is_integer: bool = rng.gen();

    // Create an Option<i32> based on the random boolean, if true:
    let value: Option<i32> = if is_integer {
        Some(rng.gen_range(1..10))  // Randomly generate an integer between 1 and 10
    } else {
        None  // Value is None
    };

    // Perform the desired operation based on the Option value
    // If value is Some, add 1 to the inner integer; otherwise, if value is None, use 0 as the result.
    let result = value.map(|x| x + 1).unwrap_or(0); 

    // Display the original value and the result
    match value {
        Some(v) => println!("Original value: {}, Result: {}", v, result),
        None => println!("Original value: None, Result: {}", result),
    }
}

main(); 

// Original value: None, Result: 0 
// Original value: 1, Result: 2
// Original value: 9, Result: 10
// Original value: None, Result: 0

Original value: None, Result: 0


this function takes an `Option<i32>` enum, and using a `match` if there’s a value inside, adds 1 to that value. If there isn’t a value inside, the function should return the None value and not attempt to perform any operations.  

In [39]:
fn plus_one(x: Option<i32>) -> Option<i32> {
    match x {
        None => None,
        Some(i) => Some(i + 1),
    }
}

let five = Some(5);
let six = plus_one(five);
let none = plus_one(None);

println!("six {:?}", six);

// Use pattern matching to print the value inside Some
match six {
    Some(value) => println!("six: {}", value),
    None => println!("six is None"),
};

// Use unwrap to directly access the value inside Some
println!("6: {}", six.unwrap());

six Some(6)
six: 6
6: 6


i32 is a signed 32-bit integer type. The arms’ patterns must cover all possibilities, for `Option<T>` that is None or Some.

In [40]:
fn plus_one(x: Option<i32>) -> Option<i32> {
    match x {
        Some(i) => Some(i + 1),
    }
}

Error: non-exhaustive patterns: `None` not covered

Using enums, we can also take special actions for a few particular values, but for all other values take one default action. This catch-all pattern meets the requirement that match must be exhaustive. Note that we have to put the catch-all arm last because the patterns are evaluated in order.

1.  `_` is a special pattern that matches any value and does not bind to that value
2.  `other` is a special pattern that matches any value and binds to that value

In [44]:
fn add_fancy_hat() {}
fn remove_fancy_hat() {}
fn move_player(num_spaces: u8) {
    println!("move {} spaces", num_spaces)
}

let dice_roll = 9;

match dice_roll {
    3 => add_fancy_hat(),
    7 => remove_fancy_hat(),
    other => move_player(other),
};

move 9 spaces


In [45]:
fn add_fancy_hat() {}
fn remove_fancy_hat() {}
fn reroll() {}

let dice_roll = 9;

match dice_roll {
    3 => add_fancy_hat(),
    7 => remove_fancy_hat(),
    _ => reroll(),
};

In [49]:
fn add_fancy_hat() {}
fn remove_fancy_hat() {}

let dice_roll = 9;

// nothing else happens on your turn if you roll anything other than a 3 or a 7.
match dice_roll {
    3 => add_fancy_hat(),
    7 => remove_fancy_hat(),
    _ => (), // We can express that by using the unit value, the empty tuple type
};

In Rust, `3u8` is a literal representing the unsigned 8-bit integer with a value of `3`, here are its 2 parts:

- `3` is the numeric value, which is 3 in this case.

- `u8` is the type suffix indicating that the literal represents an unsigned 8-bit integer.

So, `3u8` is an expression with the type u8 and a value of 3.

In Rust, `expressions` are pieces of code that produce a value, while `statements` are instructions that perform actions but do not produce a value

The `if let` takes a pattern and an expression separated by an equal sign. 

```
if let <Pattern> = variable {expression}
```

if `config_max` is of the `Some()` pattern as opposed to the `None` pattern

It works the same way as a `match`, where:

```
match variable {
    <Pattern> => expression,
    _ => (),
};
```

In this case, the pattern is Some(max), and the max binds to the value inside the Some so we can then use max in the body of the if let block in the same way we used max in the corresponding match arm and the code in the if let block isn’t run if the value doesn’t match the pattern. 

For example, these two are the same:

In [61]:
let config_max = Some(3u8);

In [62]:
match config_max {
    Some(max) => println!("The maximum is configured to be {}", max),
    _ => (),
};

The maximum is configured to be 3


In [63]:
if let Some(max) = config_max {
    println!("The maximum is configured to be {}", max);
};

The maximum is configured to be 3


In [69]:
let config_max: Option<u8> = None;

In [65]:
// explicitly annotate the type of config_max as Option<u8>, 

match config_max {
    Some(max) => println!("The maximum is configured to be {}", max),
    _ => (),
};

if let Some(max) = config_max {
    println!("The maximum is configured to be {}", max);
};

These `match` and `if let else` control flows are also both the same. 

In [71]:
let mut count = 0;

match config_max {
    Some(max) => println!("The maximum is configured to be {}", max),
    _ => count += 1,
}

println!("{}",count);

1


In [72]:
let mut count = 0;

if let Some(max) = config_max {
    println!("The maximum is configured to be {}", max);
} else {
    count += 1;
}

println!("{}",count);

1
